In [2]:
import pandas as pd
import xarray as xr
import numpy as np
import seaborn as sns
import os
import folium
import folium.plugins as plugins
from folium.features import DivIcon
from metpy.calc import dewpoint_from_relative_humidity, altimeter_to_sea_level_pressure, equivalent_potential_temperature
from metpy.units import units
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import mpld3
import branca
import base64

def reduce_to_slp(altitude, station_pressure, temp):
    rd = 287.05
    g = 9.8065
    T = temp + 273.15
    H  = (rd * T) / g
    

    mslp = station_pressure * np.exp(altitude/H)


    return mslp 

# https://gis.stackexchange.com/questions/443295/layers-categories-in-folium-and-google-earth-engine-python-api

%store -r file_name

met_data = pd.read_csv(file_name)
station_data = pd.read_csv('station_info_new.csv').rename(columns={'id':'station'})

merged_data = pd.merge(met_data, station_data, on='station')



# calculate the dewpoint and add the dataframe
td = dewpoint_from_relative_humidity(merged_data['tl'].values * units.degC, merged_data['rf'].values * units.percent)
mslp = reduce_to_slp(np.ma.masked_greater(merged_data['altitude'].values, 2000), merged_data['p'].values, merged_data['tl'].values)


merged_data['time'] = pd.to_datetime(merged_data['time'])
merged_data['td'] = td.magnitude.round(decimals=1)
merged_data['mslp'] = mslp.round(decimals=1)
merged_data['ffxkmh'] = (merged_data['ffx'] * 3.6).round(decimals=1)

thetae = equivalent_potential_temperature(merged_data['p'].values * units.hPa, merged_data['tl'].values * units.degC, merged_data['td'].values * units.degC).to('degC')
merged_data['thetae'] = thetae.magnitude.round(decimals=1)



# extract = merged_data[merged_data['name'].str.contains(r'Mariazell|Lilienfeld|St.P')][['tl','time','name', 'lat', 'lon']]
extract = (merged_data[['tl','tlmax','td','thetae','dd', 'time','ffxkmh', 'ddx','cglo','rr','p','sh','mslp','so','tlmin','ts', 'name', 'lat', 'lon', 'altitude']]).set_index('time')

extract = extract.dropna(thresh=extract.shape[1]-4, axis=0).drop_duplicates().sort_values(by=['name', 'time'])
# 24h precipitation sum
test_sum = extract.groupby(['name'])['rr'].rolling('24h').sum().round(decimals=1)
test_sum.rename('rr_sum', inplace=True)
# 12h precipitation sum
halfd_sum = extract.groupby(['name'])['rr'].rolling('12h').sum().round(decimals=1)
halfd_sum.rename('rr_sum_12', inplace=True)
# 3h precipitation sum
threeh_sum = extract.groupby(['name'])['rr'].rolling('3h').sum().round(decimals=1)
threeh_sum.rename('rr_sum_3', inplace=True)
# 1h precipitation sum
oneh_sum = extract.groupby(['name'])['rr'].rolling('1h').sum().round(decimals=1)
oneh_sum.rename('rr_sum_1', inplace=True)
# 24h max wind gust
max_gust = extract.groupby(['name'])['ffxkmh'].rolling('24h').max().round(decimals=1)
max_gust.rename('max_gust', inplace=True)
# 1h max wind gust
max_gust_one = extract.groupby(['name'])['ffxkmh'].rolling('1h').max().round(decimals=1)
max_gust_one.rename('max_gust1h', inplace=True)
# 24h max temperature
max_temp = extract.groupby(['name'])['tlmax'].rolling('24h').max().round(decimals=1)
max_temp.rename('max_temp', inplace=True)
# 24h min temperature
min_temp = extract.groupby(['name'])['tlmin'].rolling('24h').min().round(decimals=1)
min_temp.rename('min_temp', inplace=True)


extract = extract.merge(test_sum, on=['name', 'time'])
extract = extract.merge(halfd_sum, on=['name', 'time'])
extract = extract.merge(threeh_sum, on=['name', 'time'])
extract = extract.merge(oneh_sum, on=['name', 'time'])
extract = extract.merge(max_gust, on=['name', 'time'])
extract = extract.merge(max_temp, on=['name', 'time'])
extract = extract.merge(min_temp, on=['name', 'time'])
extract = extract.merge(max_gust_one, on=['name', 'time'])


individual_dates = extract.index.unique()
latest = extract.loc[extract.index == individual_dates[-2]]



latest




C:\Users\sh16450\AppData\Local\Temp\ipykernel_13024\4190056399.py:25: RuntimeWarning: overflow encountered in exp
  mslp = station_pressure * np.exp(altitude/H)


,tl,tlmax,td,thetae,dd,ffxkmh,ddx,cglo,rr,p,...,lon,altitude,rr_sum,rr_sum_12,rr_sum_3,rr_sum_1,max_gust,max_temp,min_temp,max_gust1h
time,,,,,,,,,,,,,,,,,,,,,
2024-10-26 17:10:00+00:00,11.6,11.7,9.0,39.0,81.0,6.5,12.0,0.0,0.0,936.4,...,13.343330,709.0,0.0,0.0,0.0,0.0,14.8,19.2,5.8,8.3
2024-10-26 17:10:00+00:00,17.5,17.7,7.0,45.9,155.0,37.1,147.0,0.0,0.0,911.1,...,11.706670,931.0,0.0,0.0,0.0,0.0,48.2,20.4,4.7,43.2
2024-10-26 17:10:00+00:00,11.4,11.8,9.7,40.5,46.0,5.0,54.0,0.0,0.0,931.0,...,15.240690,783.2,0.0,0.0,0.0,0.0,19.4,17.1,7.9,10.8
2024-10-26 17:10:00+00:00,11.1,11.1,9.7,38.7,231.0,4.3,224.0,0.0,0.0,943.5,...,14.138260,641.0,0.0,0.0,0.0,0.0,14.4,18.5,3.8,8.6
2024-10-26 17:10:00+00:00,10.1,10.1,9.6,38.6,198.0,11.5,199.0,0.0,0.0,933.8,...,9.848610,715.3,0.0,0.0,0.0,0.0,17.3,15.0,5.2,13.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-26 17:10:00+00:00,12.6,12.6,11.3,36.2,120.0,15.5,134.0,0.0,0.0,999.5,...,16.508889,174.0,0.0,0.0,0.0,0.0,29.5,14.2,11.7,20.9
2024-10-26 17:10:00+00:00,13.1,13.1,12.3,41.4,79.0,5.0,77.0,0.0,0.0,973.5,...,16.096780,403.6,0.0,0.0,0.0,0.0,16.9,14.0,11.1,6.5
2024-10-26 17:10:00+00:00,10.5,11.1,8.4,37.6,292.0,6.8,277.0,NaN,0.0,931.4,...,12.786390,754.0,0.0,0.0,0.0,0.0,15.8,18.8,5.3,9.7


In [3]:
def create_temp_plot(data, extreme_values=True, var='tl'):


    test = data.sort_values(by=['name', 'time']).dropna(thresh=extract.shape[1]-4, axis=0)[[var, 'lat', 'lon', 'name']]

    names_unique = test['name'].unique().tolist()
    
    fig_list, max_list, min_list = [], [], []
    lat_list, lon_list = [], []
    
    
    for name in names_unique:
        data = test.loc[test['name'] == name]
        
        

        lat_list.append(data['lat'].unique()[0])
        lon_list.append(data['lon'].unique()[0])

        fig, axs = plt.subplots(nrows=1, ncols=1)
        sns.lineplot(data=data, x='time', y=var, ax=axs, lw=4)
        sns.despine()
        fig.autofmt_xdate()
        axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

        axs.set_xlabel('Time (UTC)', fontsize=16)
        axs.set_ylabel('Temperature', fontsize=16)
        # plt.title(name)

        if extreme_values == True:
            # Find max value and index of max value along columns
            max = data.max()
            idxmax = data.idxmax(skipna=True)
            # Find min value and index of max value along columns
            min = data.min()
            idxmin = data.idxmin(skipna=True)



            axs.scatter(idxmax[var], max[var], s=100, marker='o', lw=1, zorder=3, facecolors='none', edgecolors='r')
            axs.scatter(idxmin[var], min[var], s=100, marker='o', lw=1, zorder=3, facecolors='none', edgecolors='r')



            axs.set_ylim([min[var] -2, max[var] + 2])
            
            axs.text(idxmax[var] - datetime.timedelta(hours=1.2), max[var] + 0.6, str(max[var]), fontsize=18, alpha=0.7)
            axs.text(idxmin[var] - datetime.timedelta(hours=1.5), min[var]-1.7, str(min[var]), fontsize=18, alpha=0.7)
            
            
            folder = "C:\\Users\\sh16450\\OneDrive - University of Bristol\\Documents\\GitHub\\openweatherviz\\folium\\content\\" + var + '\\'
            try:
                os.makedirs(folder)
            except FileExistsError:
                # directory already exists
                pass
            
            save_str = folder + var + '_' + name + '.png'
            
            fig.savefig(save_str)
            


            plt.close(fig)
            
            fig_list.append(save_str)
            min_list.append(min[var])
            max_list.append(max[var])
            
            print(name)
            

    return fig_list, names_unique, min_list, max_list, lat_list, lon_list


def get_popup(fig_loc, name_str, elev_str, tl_max_str, tl_min_str):
    encoded = base64.b64encode(open(fig_loc, 'rb').read()).decode()
    html = f'''
        <h1>Station: {name_str}</h1>
        <p style="font-size:14px; color:#000000; font-weight:bold;">
        Elevation: {elev_str} m
        <br />
        <span style="color:red">Tmax: {tl_max_str}&deg;C<span/>
        <br />
        <span style="color:blue">Tmin: {tl_min_str}&deg;C<span/>
        </p>
        <img src="data:image/png;base64,{encoded}">'''
    iframe = folium.IFrame(html, width=650, height=650)
    popup = folium.Popup(iframe)
    
    return popup, html

#WIP CONTINUE HERE
def create_popup_and_html_list(fig_list, names, min_list, max_list, lat_list, lon_list, data):

    dict = {'name': names, 'figure_path': fig_list, 'min_var': min_list, 'max_var': max_list, 'lat': lat_list, 'lon': lon_list} 
    df_extremes = pd.DataFrame(dict)
    df_extremes = df_extremes.merge(data, on=['name', 'lat', 'lon'])

    # get the popups for tthe variable
    popup_list, html_list = [], []
    for i in range(np.shape(df_extremes)[0]):
        popup, html = get_popup(df_extremes['figure_path'][i], df_extremes['name'][i], df_extremes['altitude'][i],df_extremes['max_var'][i],df_extremes['min_var'][i])
        popup_list.append(popup)
        html_list.append(html)
        
    dict_two = {'html': html_list, 'popup': popup_list, 'name': df_extremes['name'].values.tolist()}
    df_two = pd.DataFrame(dict_two)
    
    output_df = df_extremes.merge(df_two, on='name')
    
    return df_extremes, df_two, output_df




fig_list, names, min_list, max_list, lat_list, lon_list = create_temp_plot(extract, extreme_values=True, var='tl')
df_extremes, df_two, output_df = create_popup_and_html_list(fig_list, names, min_list, max_list, lat_list, lon_list, latest)

Abtenau
Achenkirch
Aflenz
Aigen im Ennstal
Alberschwende
Allentsteig
Alpbach
Altmünster
Amstetten
Andau
Arriach
Aspach
Aspang
Bad Aussee
Bad Bleiberg
Bad Eisenkappel
Bad Gastein
Bad Gleichenberg
Bad Goisern
Bad Ischl
Bad Mitterndorf
Bad Radkersburg
Bad Zell
Berndorf
Bernstein
Bischofshofen
Bludenz
Brand
Bregenz
Brenner
Bruck an der Mur
Bruckneudorf
Brunn am Gebirge
Buchberg
Bärnkopf
Dellach
Deutschlandsberg
Dornbirn
Döllach
Ehrwald
Eisenstadt Nordost
Enns
Feistritz ob Bleiburg
Feldbach
Feldkirch
Feldkirchen
Ferlach
Feuerkogel
Fischbach
Flattnitz
Fraxern
Freistadt
Fresach
Friesach
Frohnleiten
Fürstenfeld
Galtür
Galzig
Gars am Kamp
Gaschurn
Gleisdorf
Gmünd
Golling
Graz Straßgang
Graz Thalerhof Flughafen
Groß-Enzersdorf
Gröbming
Gumpoldskirchen
Gutenstein Mariahilfberg
Gänserndorf Stadt
Güssing
Haiming
Hall bei Admont
Hartberg
Hermagor
Hochfilzen
Hohe Wand Hochkogelhaus
Hohe Wand-Hochkogelhaus
Hohenau
Holzgau
Horn Wasserwerk
Imst
Innsbruck Flughafen
Innsbruck Universität
Irdning-Gumpenste

In [30]:
output_df

,name,figure_path,min_var,max_var,lat,lon,tl,tlmax,td,thetae,...,rr_sum,rr_sum_12,rr_sum_3,rr_sum_1,max_gust,max_temp,min_temp,max_gust1h,html,popup
0,Abtenau,C:\Users\sh16450\OneDrive - University of Bris...,2.3,5.6,47.566670,13.343330,3.0,3.0,2.3,21.2,...,86.3,48.5,13.5,6.1,52.6,5.7,2.2,13.3,\n <h1>Station: Abtenau</h1>\n <...,<folium.map.Popup object at 0x00000223983E4B60>
1,Achenkirch,C:\Users\sh16450\OneDrive - University of Bris...,0.7,3.8,47.499440,11.706670,3.1,3.3,2.4,23.8,...,41.2,16.5,2.9,0.7,35.6,3.8,0.7,35.6,\n <h1>Station: Achenkirch</h1>\n ...,<folium.map.Popup object at 0x00000223971C9160>
2,Aflenz,C:\Users\sh16450\OneDrive - University of Bris...,4.5,8.1,47.545940,15.240690,5.0,5.1,2.4,25.3,...,14.8,14.8,9.3,3.2,46.4,8.1,4.4,34.6,\n <h1>Station: Aflenz</h1>\n <p...,<folium.map.Popup object at 0x00000223987D7FE0>
3,Aigen im Ennstal,C:\Users\sh16450\OneDrive - University of Bris...,4.4,6.9,47.532780,14.138260,5.3,5.8,2.3,23.3,...,39.2,28.8,8.3,2.0,64.8,6.9,4.4,61.6,\n <h1>Station: Aigen im Ennstal</h1>\n...,<folium.map.Popup object at 0x00000223E77AB080>
4,Alberschwende,C:\Users\sh16450\OneDrive - University of Bris...,4.5,8.0,47.457500,9.848610,6.7,6.7,6.4,29.2,...,17.2,5.9,1.4,0.8,37.4,8.2,4.5,11.2,\n <h1>Station: Alberschwende</h1>\n ...,<folium.map.Popup object at 0x00000223E2DE1250>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,Wolkersdorf,C:\Users\sh16450\OneDrive - University of Bris...,7.2,9.1,48.388056,16.508889,8.5,8.5,7.9,28.2,...,52.0,43.6,15.5,3.5,67.0,9.1,7.2,55.4,\n <h1>Station: Wolkersdorf</h1>\n ...,<folium.map.Popup object at 0x00000223987D7500>
257,Wörterberg,C:\Users\sh16450\OneDrive - University of Bris...,7.2,11.4,47.226190,16.096780,8.8,8.8,4.2,27.2,...,7.1,6.9,2.0,0.3,65.2,11.4,7.2,64.4,\n <h1>Station: Wörterberg</h1>\n ...,<folium.map.Popup object at 0x000002239876B980>
258,Zell am See,C:\Users\sh16450\OneDrive - University of Bris...,4.1,5.9,47.290000,12.786390,4.6,4.6,2.5,23.6,...,18.3,12.2,2.4,0.4,43.6,5.9,4.0,33.8,\n <h1>Station: Zell am See</h1>\n ...,<folium.map.Popup object at 0x0000022397C3AB40>
259,Zeltweg,C:\Users\sh16450\OneDrive - University of Bris...,5.7,9.7,47.199830,14.746890,5.9,6.1,3.1,25.6,...,3.2,3.2,3.2,0.8,64.1,9.9,5.7,41.0,\n <h1>Station: Zeltweg</h1>\n <...,<folium.map.Popup object at 0x0000022398BBA690>


In [4]:


m = folium.Map(location=[47.73, 13.48], zoom_start=8, tiles=None)
base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='https://tile.jawg.io/jawg-dark/{z}/{x}/{y}{r}.png?access-token=MglwGpuT1VUaVc2LSRX042X3krQz6y2cOZfGtNqWg3VMUke6gemUIWz7G1uxPPXP',
                 attr='CartoDB.Voyager', max_zoom=20).add_to(base_map)
base_map.add_to(m)

# Add the date as a title to the map
map_title = individual_dates[-2].strftime("%Y-%m-%d   %H:%M UTC")
title_html = f'<h1 style="position:absolute;z-index:100000;left:10vw;background-color:#eeeee4;" >{map_title}</h1>'
m.get_root().html.add_child(folium.Element(title_html))


COLORS = [
    (lambda x: x > 40, "#EC8989"),
    (lambda x: 35 < x <= 40, "#EA3D09"),
    (lambda x: 30 < x <= 35, "#EA3D09"),
    (lambda x: 25 < x <= 30, "#EA6509"),
    (lambda x: 20 < x <= 25, "#C5C71B"),
    (lambda x: 15 < x <= 20, "#8EF700"),
    (lambda x: 10 < x <= 15, "#07AF02"),
    (lambda x: 5 < x <= 10, "#94EA7F"),
    (lambda x: 0 < x <= 5, "#9cecc8"),
    (lambda x: 0 >= x >= -5, "#18B5DA"),
    (lambda x: -5 > x >= -10, "#1661e8"),
    (lambda x: -10 > x >= -15, "#A375E6"),
    (lambda x: -15 > x >= -20, "#ce8bf0"),
    (lambda x: x < -20, "#EC9CE6"),
]

COLORS_WIND = [
    (lambda x: x >= 100, "#EF06E1"), # purple pizazz
    (lambda x: 80 <= x < 100, "#EF0620"), # torch red
    (lambda x: 60 <= x < 80, "#FB5D04"), # blaze orange
    (lambda x: 40 <= x < 60, "#D8DC0F"), # bitter lemon
    (lambda x: 30 <= x < 40, "#4FBF2A"), # apple
    (lambda x: x < 30, "#000000"), # black
    
    
]

COLORS_PREC_SUM = [
    (lambda x: x >= 80, "#EF06E1"), # purple pizazz
    (lambda x: 50 <= x < 80, "#EF0620"), # torch red
    (lambda x: 30 <= x < 50, "#FB5D04"), # blaze orange
    (lambda x: 10 <= x < 30, "#D8DC0F"), # bitter lemon
    (lambda x: 1 <= x < 10, "#4FBF2A"), # apple
    (lambda x: x < 1, "#000000"), # black
    
    
]




# This picks the color for temperature and tmax, dewpoint and wind
def get_color(x, color_list):
    for predicate, color in color_list:
        if predicate(x):
            return color

    return 'black'



# Add analysis for temperature
fg = folium.FeatureGroup(name="Temperature (last 10 min)", control=True, overlay=False).add_to(m)
# marker_cluster = plugins.MarkerCluster(options={'disableClusteringAtZoom':10, 'maxClusterRadius':5, 'singleMarkerMode':False}).add_to(m)

for i in range(np.shape(output_df)[0]):
        color = get_color(output_df['tl'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(output_df['tl'].iloc[i]) + '</div>'
        p1 = [output_df['lat'].iloc[i], output_df['lon'].iloc[i]]
        folium.Marker(p1,popup=output_df['popup'][i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fg)

        # fg.add_child(popup_list[i])
 
 
# Add analysis for maximum temperature        
fgg = folium.FeatureGroup(name="Tmax (last 10 min)", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(latest)[0]):
        color = get_color(latest['tlmax'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(latest['tlmax'].iloc[i]) + '</div>'
        p1 = [latest['lat'].iloc[i], latest['lon'].iloc[i]]
        folium.Marker(p1,popup=latest['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fgg)
        


# Add analysis for dewpoint temperature        
fggg = folium.FeatureGroup(name="Dewpoint (last 10 min)", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(latest)[0]):
        color = get_color(latest['td'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(latest['td'].iloc[i]) + '</div>'
        p1 = [latest['lat'].iloc[i], latest['lon'].iloc[i]]
        folium.Marker(p1,popup=latest['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fggg)
        
# Add analysis for ThetaE        
fgggg = folium.FeatureGroup(name="Eq. potential Temperature (last 10 min)", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(latest)[0]):
        # use temperature color but reduce by 20C
        color = get_color(latest['thetae'].iloc[i] - 20, COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(latest['thetae'].iloc[i]) + '</div>'
        p1 = [latest['lat'].iloc[i], latest['lon'].iloc[i]]
        folium.Marker(p1,popup=latest['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fgggg)

           
# Add analysis for 24h maximum temperature        
tm = folium.FeatureGroup(name="24h Maximum Temperature", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(output_df)[0]):
        color = get_color(output_df['max_temp'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(output_df['max_temp'].iloc[i]) + '</div>'
        p1 = [output_df['lat'].iloc[i], output_df['lon'].iloc[i]]
        folium.Marker(p1,popup=output_df['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(tm)

     
# Add analysis for 24h minimum temperature        
tmm = folium.FeatureGroup(name="24h Minmum Temperature", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(output_df)[0]):
        color = get_color(output_df['min_temp'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(output_df['min_temp'].iloc[i]) + '</div>'
        p1 = [output_df['lat'].iloc[i], output_df['lon'].iloc[i]]
        folium.Marker(p1,popup=output_df['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(tmm)
        
        
# Add analysis for 24h precipitation        
pr = folium.FeatureGroup(name="24h Precipitation Sum", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(output_df)[0]):
        color = get_color(output_df['rr_sum'].iloc[i], COLORS_PREC_SUM)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(output_df['rr_sum'].iloc[i]) + '</div>'
        p1 = [output_df['lat'].iloc[i], output_df['lon'].iloc[i]]
        folium.Marker(p1,popup=output_df['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(pr)
        
# Add analysis for 12h precipitation        
prr = folium.FeatureGroup(name="12h Precipitation Sum", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(output_df)[0]):
        color = get_color(output_df['rr_sum_12'].iloc[i] * 1.5, COLORS_PREC_SUM)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(output_df['rr_sum_12'].iloc[i]) + '</div>'
        p1 = [output_df['lat'].iloc[i], output_df['lon'].iloc[i]]
        folium.Marker(p1,popup=output_df['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(prr)
        
# Add analysis for 3h precipitation        
prrr = folium.FeatureGroup(name="3h Precipitation Sum", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(output_df)[0]):
        color = get_color(output_df['rr_sum_3'].iloc[i] * 4, COLORS_PREC_SUM)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(output_df['rr_sum_3'].iloc[i]) + '</div>'
        p1 = [output_df['lat'].iloc[i], output_df['lon'].iloc[i]]
        folium.Marker(p1,popup=output_df['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(prrr)

# Add analysis for 1h precipitation        
prrr = folium.FeatureGroup(name="1h Precipitation Sum", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(output_df)[0]):
        color = get_color(output_df['rr_sum_1'].iloc[i] * 10, COLORS_PREC_SUM)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(output_df['rr_sum_1'].iloc[i]) + '</div>'
        p1 = [output_df['lat'].iloc[i], output_df['lon'].iloc[i]]
        folium.Marker(p1,popup=output_df['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(prrr)
        



# Add analysis for wind direction of max gust and gust direction
ddg = folium.FeatureGroup(name="Wind direction (max gust) + Max Gust (kmh) - last 10 min", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(latest)[0]):
    rotate_str = str(latest['ddx'].iloc[i] + 90)
    color = get_color(latest['ffxkmh'].iloc[i], COLORS_WIND)
    folium.Marker(location=[latest['lat'].iloc[i], latest['lon'].iloc[i]], popup=latest['name'].iloc[i],
                            icon=plugins.BeautifyIcon(icon="fa-solid fa-arrow-right",
                                            border_color='transparent',
                                            background_color='transparent',
                                            border_width=1,
                                            text_color=color,
                                            inner_icon_style='margin:0px;font-size:24px;transform: rotate({0}deg);'.format(rotate_str))).add_to(ddg)
    
# Add analysis for max wind gust        
ddgg = folium.FeatureGroup(name="Max Gust 24h (kmh)", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(output_df)[0]):
        color = get_color(output_df['max_gust'].iloc[i], COLORS_WIND)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(output_df['max_gust'].iloc[i]) + '</div>'
        p1 = [output_df['lat'].iloc[i], output_df['lon'].iloc[i]]
        folium.Marker(p1,popup=output_df['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(ddgg)
        
# Add analysis for 1h max wind gust        
ddgg = folium.FeatureGroup(name="Max Gust 1h (kmh)", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(output_df)[0]):
        color = get_color(output_df['max_gust1h'].iloc[i], COLORS_WIND)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(output_df['max_gust1h'].iloc[i]) + '</div>'
        p1 = [output_df['lat'].iloc[i], output_df['lon'].iloc[i]]
        folium.Marker(p1,popup=output_df['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(ddgg)

folium.LayerControl().add_to(m)

plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)


# icon = folium.DivIcon(html="17.4")
# folium.Marker([47.0, 15.0], icon=icon).add_to(m)
m.save(os.path.join('simple_popups.html'))

In [10]:
output_df['sh'].nlargest(n=5)

60     67.0
168    58.0
255    34.0
96     26.0
204    20.0
Name: sh, dtype: float64

In [89]:
latest

,tl,tlmax,td,dd,ffxkmh,ddx,cglo,rr,p,sh,mslp,so,tlmin,ts,name,lat,lon,altitude,rr_sum
time,,,,,,,,,,,,,,,,,,,
2024-09-12 19:50:00+00:00,1.3,1.3,0.7,179.0,5.4,191.0,0.0,0.2,933.5,NaN,1019.6,0.0,1.2,1.2,Abtenau,47.566670,13.343330,709.0,34.9
2024-09-12 19:50:00+00:00,5.2,5.8,4.2,4.0,15.5,267.0,0.0,0.0,907.5,NaN,1017.4,0.0,5.2,5.4,Achenkirch,47.499440,11.706670,931.0,12.2
2024-09-12 19:50:00+00:00,4.8,4.8,4.1,52.0,7.6,58.0,0.0,0.1,922.6,NaN,1015.8,0.0,4.7,5.6,Aflenz,47.545940,15.240690,783.2,28.0
2024-09-12 19:50:00+00:00,1.1,1.1,0.7,347.0,4.0,330.0,0.0,0.1,939.7,NaN,1017.8,0.0,1.0,1.3,Aigen im Ennstal,47.532780,14.138260,641.0,41.2
2024-09-12 19:50:00+00:00,7.3,7.3,4.8,156.0,4.7,141.0,0.0,0.0,931.3,NaN,1016.1,0.0,7.2,8.0,Alberschwende,47.457500,9.848610,715.3,10.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-12 19:50:00+00:00,8.0,8.0,6.6,337.0,27.7,339.0,0.0,0.1,990.8,NaN,1012.0,0.0,8.0,7.8,Wolkersdorf,48.388056,16.508889,174.0,11.7
2024-09-12 19:50:00+00:00,7.4,7.4,6.3,286.0,9.4,255.0,0.0,0.0,963.0,NaN,1011.5,0.0,7.3,8.9,Wörterberg,47.226190,16.096780,403.6,19.6
2024-09-12 19:50:00+00:00,0.4,0.5,-0.0,270.0,5.4,266.0,NaN,0.2,928.0,NaN,1019.6,0.0,0.4,NaN,Zell am See,47.290000,12.786390,754.0,40.1
